# PANTHER API access and gene annotation

Functional annotation of protein families in Differentially Expressed Genes and Leading Edge list in Enrichment result

_Author : Natalia García Sánchez_

_Date : 25/05/2023_

---

In [1]:
import requests
import pandas as pd

directory = input("Please enter a directory to output PANTHERdb annotations")

current panther release

In [17]:
request_url = "http://pantherdb.org/services/oai/pantherdb/supportedgenomes"
res = requests.post(request_url, {'caller_identity':'https://github.com/Natpod/Bnapus_ME'})

In [ ]:
version=res.json()['search']['product']


In [29]:
version

{'source': 'PANTHERDB', 'version': 17}

In [33]:
gen=res.json()['search']['output']['genomes']['genome']
supportedGenomes = pd.DataFrame.from_dict(gen)

In [36]:
supportedGenomes[supportedGenomes['taxon_id']==3708]

,name,taxon_id,short_name,version,long_name
26,canola,3708,BRANA,Reference Proteome 2021_03,Brassica napus


download genome annotations POST (only 1000 genes at a time)

In [41]:
start_index=1
## Construct the request
##
request_url = f"http://pantherdb.org/services/oai/pantherdb/downloadgenome?organism=3708&startIndex={start_index}"

params = {
    "caller_identity": "https://github.com/Natpod/Bnapus_ME",
}
results = requests.post(request_url, data=params)
#df = handle_results(results)

In [43]:
response = results.json()

In [46]:
print("Number of genes found for protein functional annotation:", response['search']['number_of_genes_in_genome'])

Number of genes found for protein functional annotation: 60104


In [313]:
final_index = response['search']['number_of_genes_in_genome']
max_genes_retrieved = 1000
num_of_requests = int(round((final_index-start_index)/max_genes_retrieved,0))

Do for all genes

In [323]:
final_df = pd.DataFrame(columns = [['gene_symbol','gene_name','persistent_id','name_PC','id_PC']])

start_index=1
final_index = response['search']['number_of_genes_in_genome']
max_genes_retrieved = 1000
num_of_requests = int(round((final_index-start_index)/max_genes_retrieved,0))

indexes_to_test = [1]
indexes_to_test = indexes_to_test + [i*1000 for i in range(1,num_of_requests+1)]

for st_index_request in indexes_to_test:
    ## Construct the request
    ##
    request_url = f"http://pantherdb.org/services/oai/pantherdb/downloadgenome?organism=3708&startIndex={st_index_request}"

    params = {
        "caller_identity": "https://github.com/Natpod/Bnapus_ME",
    }
    results = requests.post(request_url, data=params)






    gl_1 = pd.DataFrame.from_dict(response['search']['gene_list']['gene'])
    gl_1 = gl_1.replace(nanval,None)

    list_genes = gl_1['gene_symbol'].to_list()
    list_gene_names = gl_1['gene_name'].to_list()
    list_gene_pantherid = gl_1['persistent_id'].to_list()
    list_name_pc= []
    list_id_pc = []

    for i in range(len(list_annotations)):
        if list_annotations[i] != None:
            tmp_df = pd.DataFrame.from_dict(list_annotations[i]['annotation_data_type'])
            if tmp_df.shape[0] > 1 :
                if (tmp_df['content'] == 'ANNOT_TYPE_ID_PANTHER_PC').any():
                    tmp_df = tmp_df[tmp_df['content'] == 'ANNOT_TYPE_ID_PANTHER_PC']['annotation_list']
                    dict_pc = tmp_df.iloc[0]
                    #subdict = dict_pc.get('annotation')
                    name_pc = dict_pc.get('annotation').get('name')
                    id_pc = dict_pc.get('annotation').get('id')
            else:
                if (tmp_df['content'] == 'ANNOT_TYPE_ID_PANTHER_PC').any():
                    tmp_df = tmp_df[tmp_df['content'] == 'ANNOT_TYPE_ID_PANTHER_PC']['annotation_list']
                    name_pc = tmp_df.iloc[0].get('name')
                    id_pc = tmp_df.iloc[0].get('id')

            list_name_pc.append(name_pc)
            list_id_pc.append(id_pc)
        else:
            list_name_pc.append(None)
            list_id_pc.append(None)


    prepr_df = pd.DataFrame(list(zip(list_genes,list_gene_names,list_gene_pantherid,list_name_pc,list_id_pc)), columns = [['gene_symbol','gene_name','persistent_id','name_PC','id_PC']])
    final_df = pd.concat((final_df, prepr_df))

In [324]:
final_df.shape

(61000, 5)

Get _Brassica napus_ Hidden Markov Model Based assignation of PANTHERdb protein family classification on ftp browser on linux command (current PANTHER release version : v.17).The assignment is based on the phylogenetic trees where the gene is located.  The annotation of the subfamilies are used for the gene.

```
wget 'http://data.pantherdb.org/ftp/sequence_classifications/17.0/PANTHER_Sequence_Classification_files/PTHR17.0_rape'

```



In [343]:
df_Fam = pd.read_csv('/home/famgarcia/Descargas/PTHR17.0_rape', sep="\t", header=None)

df_Fam.rename(columns=dict(zip(df_Fam.columns,['gene_id','Uniprot_ID','gene_symbol','PANTHER_SF_ID','PANTHER_Family_name','PANTHER_SF_name','GO_MF','GO_BP','GO_CC','Protein_class','Pathway'])), inplace=True)
df_Fam = df_Fam[['Uniprot_ID','gene_symbol','PANTHER_SF_ID','PANTHER_Family_name','PANTHER_SF_name']]

In [338]:
df_Fam.head()

,Uniprot_ID,gene_symbol,PANTHER_SF_ID,PANTHER_Family_name,PANTHER_SF_name
0,A0A078JWU9,BnaAnng36540D,PTHR31929:SF143,SAUR-LIKE AUXIN-RESPONSIVE PROTEIN FAMILY-RELATED,BNAANNG36540D PROTEIN
1,A0A078IGF9,BnaC05g48700D,PTHR45621:SF151,OS01G0588500 PROTEIN-RELATED,PROTEIN KINASE DOMAIN-CONTAINING PROTEIN
2,A0A078FKQ7,BnaCnng04230D,PTHR21649:SF120,CHLOROPHYLL A/B BINDING PROTEIN,"CHLOROPHYLL A-B BINDING PROTEIN, CHLOROPLASTIC"
3,A0A078H4L9,BnaA06g35530D,PTHR15680:SF16,RIBOSOMAL PROTEIN L19,BNAA06G35530D PROTEIN
4,A0A078JN16,BnaUnng02600D,PTHR11945:SF688,MADS BOX PROTEIN,MADS-BOX DOMAIN-CONTAINING PROTEIN


In [346]:
final_df.head()

final_df.columns = final_df.columns.map(''.join)

merge both results

In [347]:
df_annotated = pd.merge(final_df, df_Fam, on="gene_symbol",how="outer")

In [353]:
df_annotated = df_annotated.drop_duplicates()

In [379]:
df_annotated = df_annotated.replace(nanval,None)
df_annotated.to_csv(directory+"\PANTHER_proteins.tsv", sep="\t", index=False)